# MSA 2023 Phase 2 - Part 2

In [10]:
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Load and split preprocessed data

In [24]:
MarketData = pd.read_csv('PreprocessedMarketData.csv')
MarketData = MarketData.drop('Unnamed: 0', axis=1)
MarketData.head()

,Is_Male,Ever_Married,Age,Graduated,Work_Experience,Spending_Score,Family_Size,Segmentation,Profession_Artist,Profession_Doctor,...,Profession_Homemaker,Profession_Lawyer,Profession_Marketing,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
0,1,0,0.247191,0,0.071429,1,0.444444,D,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1,0.752809,1,0.071429,1,0.111111,B,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,1,0.752809,1,0.000000,3,0.222222,B,0,0,...,0,1,0,0,0,0,0,0,1,0
3,1,1,0.629213,0,0.000000,2,0.222222,C,1,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0.359551,1,0.071429,1,0.333333,C,0,0,...,0,0,0,0,0,0,0,0,1,0


The data consists of the following   
boolean values (1 for yes, 0 for no) for: Is_Male, Ever_Married, Graduated, all of the professions, and the values of 'Var_1'.
The last two are from one-hot vectors meaning for each type only one value is true, all of these names are self describing, apart from Var_1 which I do not understand    
As well as this there are numerical values which have been normalised for: Age, Work_Experience and Family_Size    
The market segments are a categorical value from A through D    
The spending score is ordinal data, with 1 being low, 2 being average, and 3 being high


In [25]:
X = MarketData.drop('Segmentation', axis=1)
y = MarketData['Segmentation']

In [26]:
from sklearn.model_selection import train_test_split
  # Create an 70-30 train-test split on the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)



## 2. Choose an algorithm

In [14]:
#

## 3. Train and test a model

In [15]:
#

## 4. Evaluate the model 

In [16]:
#

## 5. Summary